In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
import random

In [5]:
!pip install mysql-connector-python

<h1> get data from database</h1>

In [2]:
import mysql.connector
from mysql.connector import Error
records=[]
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='optimatets',
                                         user='root',
                                         password='')
    sql_select_Query=[]
    sql_select_Query.append( "select * from bookings")
    sql_select_Query.append("select * from driver_vehicle")
    sql_select_Query.append("select * from tracking")
    sql_select_Query.append("select * from vehicles")
    sql_select_Query.append("select * from vehicle_types")
    for req in sql_select_Query:
        cursor = connection.cursor()
        cursor.execute(req)
        records.append( cursor.fetchall())    

except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
        print("MySQL connection is closed")

MySQL connection is closed


In [3]:
bookings=pd.DataFrame(records[0],columns=['id', 'book_type', 'promo_id' , 'type_id' , 'accept_status', 'total_weight', 'company_id', 'date_time', 'user_id', 'vehicle_id', 'driver_id', 'customer_id', 'pickup', 'dropoff', 'duration', 'pickup_addr', 'dest_addr','note', 'travellers', 'status', 'payment', 'created_at', 'updated_at', 'deleted_at'])
bookings.head()

,id,book_type,promo_id,type_id,accept_status,total_weight,company_id,date_time,user_id,vehicle_id,...,duration,pickup_addr,dest_addr,note,travellers,status,payment,created_at,updated_at,deleted_at
0,4,s,NaN,17,53,30.0,11.0,2019-07-03 12:01:13,53,NaN,...,None,"Tunis, Tunisia","Sousse, Tunisia",None,1,0,0,2019-07-01 11:02:24,2019-07-29 12:20:08,2019-07-29 12:20:08
1,8,s,NaN,17,53,20.0,11.0,2019-07-06 07:09:11,53,NaN,...,None,"Cafe Karawen, Route Gremda, Sfax, Tunisia","Avenue De Carthage, Tunisia",None,1,0,0,2019-07-04 14:13:04,2019-07-29 12:20:16,2019-07-29 12:20:16
2,9,s,NaN,6,60,10.0,11.0,2019-07-08 08:21:49,60,7.0,...,None,"tunis, Tunisie","Bizerte Centre, Rue Mohamed Ali, Bizerte, Tunisia",None,1,0,0,2019-07-04 14:23:12,2019-07-29 12:20:25,2019-07-29 12:20:25
3,14,s,NaN,3,51,5.0,10.0,2019-07-11 09:06:34,51,3.0,...,None,"Avenue De Carthage, Tunisie","Sahloul, Sousse, Tunisie",Bien reçu. Je note çà appele moi sur 22222222,1,3,1,2019-07-09 11:08:28,2019-07-29 12:21:40,2019-07-29 12:21:40
4,15,s,NaN,17,60,25.0,11.0,2019-07-14 12:24:05,60,7.0,...,None,"Bizerte, Tunisie","Mahdia, Tunisie",None,1,1,0,2019-07-09 11:24:51,2019-07-29 12:21:47,2019-07-29 12:21:47


In [4]:
df1=bookings
df1.shape

(197, 24)

<h1>add some data to data base </h1>
<h4>to improve the model we added some random rows . so , we picked a random row and we changeed the date  </h4>

In [5]:
def  maxDate(df):
    lista=[]
    for i in range(len(df['date_time'])):
        lista.append(datetime.timestamp(df['date_time'][i]))
    return max(lista)

def addRow(df1,date_time):
    row = df1.sample()
    row['date_time']=date_time
    df1=df1.append(row)
    return df1
            

def fillDatabase(df1):
    now = datetime.timestamp(datetime.now())
    max_date=maxDate(df1)+ 76522
    day = max_date
    cpt=0
    while now> day :
        day=day+86400
        max_date=day+random.randint(39600,86339)
        df1=addRow(df1,datetime.fromtimestamp(max_date))
        cpt=cpt+1
        
        for i in range(random.randint(0,5)):
            max_date=day+random.randint(21600,39600)
            #print(datetime.fromtimestamp(max_date))
            df1=addRow(df1,datetime.fromtimestamp(max_date))
            cpt=cpt+1
            
            for j in range(random.randint(0,3)):
                max_date=day+random.randint(0,21600)
                #print(datetime.fromtimestamp(max_date))
                df1=addRow(df1,datetime.fromtimestamp(max_date))
                cpt=cpt+1
                
    print(cpt ,"rows added !")        
   
                    
                
       
    return df1
   
   
  

In [6]:
df1=fillDatabase(df1)  
df1.shape

1851 rows added !


(2048, 24)

<h1> adress data normalisation <h1>

In [7]:

region=['Tunis','Manouba','ben arous',
        'ariana',' Bizerte','beja', 'jandouba', 'nabeul' , 'zaghouane', 'silana', 'kef' ,
        'kasserine' , 'kairouane' , 'Sousse' , 'monastir', 'Mahdia' , 'Sfax' , 'sidi' 'bouzid' , 'gafsa' ,
        'touzeur', 'kbeli' , 'gabes' , 'mednine' ,'tataouine']
region=[x.upper() for x in region]
destination_adress= df1['dest_addr'] 
depart_adress = df1['pickup_addr']

In [8]:

def adressNormalisation(col):
    i=0
    lista=[]
    adr=''
    for phrase in col:
        verif=False
        for adrs in region:
            if adrs in phrase.upper():
                verif=True
                adr=adrs
        if verif==True :
                #lista.append([i,phrase,adr])
                lista.append([adr])

        else : 
                #lista.append([i,phrase,random.choice(region)])
                lista.append([random.choice(region)])


        i=i+1


    return pd.DataFrame(lista, columns=['new_'+col.name])



In [9]:

def adressToNan(col):
    i=0
    lista=[]
    adr=''
    for phrase in col:
        verif=False
        for adrs in region:
            if adrs in phrase.upper():
                verif=True
                adr=adrs
        if verif==True :
                #lista.append([i,phrase,adr])
                lista.append([adr])

        else : 
                #lista.append([i,phrase,random.choice(region)])
                lista.append([np.nan])


        i=i+1


    return pd.DataFrame(lista, columns=['new_'+col.name])



In [10]:
df1['pickup_addr']=adressNormalisation(df1['pickup_addr'])
df1['dest_addr']=adressNormalisation(df1['dest_addr'])

<h1> fill the nan vehicle_id </h1>
<h4>this method return a random vihicle_id by adress </h4>

In [11]:
df2=df1
def getVihicleByAdress(df2, adress):
    df2=df2.dropna(subset=["vehicle_id"])
    df2=df2.groupby('pickup_addr')['vehicle_id'].apply(list)
    x=list(set(df2[adress]))
    return random.choice(x)
 
getVihicleByAdress(df2,'SFAX')



21.0

In [12]:
df2.groupby('pickup_addr')['vehicle_id'].apply(list)

pickup_addr
ARIANA     [8.0, 8.0, nan, 8.0, 8.0, 8.0, 8.0, nan, 8.0, ...
BEJA       [23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23....
GABES      [nan, nan, nan, nan, 21.0, nan, nan, nan, nan,...
MANOUBA                            [5.0, 5.0, 5.0, 5.0, 5.0]
SFAX       [nan, 9.0, nan, nan, nan, 10.0, nan, 10.0, 9.0...
SOUSSE     [nan, nan, nan, 25.0, 21.0, nan, 21.0, 22.0, n...
TUNIS      [nan, 7.0, 3.0, 7.0, nan, nan, nan, 13.0, nan,...
Name: vehicle_id, dtype: object

In [13]:
for i in range(len(df1['vehicle_id'])):
    if np.isnan(df1['vehicle_id'].iloc[i]):
        df1['vehicle_id'].iloc[i]=getVihicleByAdress(df1,df1['pickup_addr'].iloc[i])

C:\Users\meher\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


<h1> prepare train and test data </h1>

In [14]:
vihicule_id=df1['vehicle_id'].unique()
vihicule_id

array([20., 19.,  7.,  3.,  9., 51., 21., 10., 23., 52., 13., 46., 16.,
       18., 11.,  5., 17., 22., 25.,  8.])

In [15]:
df = pd.DataFrame()
df['Datetime'] = df1['date_time']
for i in range(len(df1)):
    
    df['Year'] = df.Datetime.dt.year
    df['Month'] = df.Datetime.dt.month
    df['Day'] = df.Datetime.dt.day
    df['Hour'] = df.Datetime.dt.hour
    #df['Minute'] = df.Datetime.dt.minute


In [16]:
df1.shape

(2048, 24)

In [17]:
y=df1[['dest_addr','pickup_addr']]

In [18]:
df1[['pickup_addr','dest_addr']].tail()

,pickup_addr,dest_addr
154,SOUSSE,SFAX
92,SFAX,TUNIS
115,SOUSSE,TUNIS
121,SOUSSE,SFAX
83,TUNIS,SFAX


In [19]:
x=pd.concat([df[['Year','Day','Month','Hour']],df1['vehicle_id']],sort=False,axis=1)


In [20]:
x.shape

(2048, 5)

In [21]:
x

,Year,Day,Month,Hour,vehicle_id
0,2019,3,7,12,20.0
1,2019,6,7,7,19.0
2,2019,8,7,8,7.0
3,2019,11,7,9,3.0
4,2019,14,7,12,7.0
...,...,...,...,...,...
154,2020,31,10,16,22.0
92,2020,31,10,13,21.0
115,2020,31,10,7,22.0
121,2020,31,10,7,21.0


In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)


<h1>train model</h1>

In [23]:

# Compare Algorithms

import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
# load dataset

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
scores = []
scoring = 'accuracy'

for name, model in models:
  
    cv_results = MultiOutputClassifier(model).fit(x_train,y_train)
   
    results.append(cv_results)
    names.append(name)
    msg = "the score of %s  is : %s " % (name, cv_results.score(x_test,y_test))
    scores.append(cv_results.score(x_test,y_test))
    print(msg)
# boxplot algorithm comparison



C:\Users\meher\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\meher\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

the score of LR  is : 0.11363636363636363 
the score of LDA  is : 0.11363636363636363 
the score of KNN  is : 0.3474025974025974 
the score of CART  is : 0.38636363636363635 
the score of NB  is : 0.17532467532467533 
the score of SVM  is : 0.06818181818181818 


In [24]:
import pickle


In [25]:
import pickle

with open("models.dat", "rb") as f:
    results=pickle.load(f)
   

In [26]:
def get_model(model):
    models = {
        'LR': 0,
        'LDA': 1,
        'KNN': 2,
        "DT": 3,
        'GNB': 4,
        'SVM': 5,
    }
    return (models.get(model, "Invalid model")) 

In [27]:

import mysql.connector
from mysql.connector import Error

def get_vehicle(ids,model_name):

    try:
        connection = mysql.connector.connect(host='localhost',
                                         database='optimatets',
                                         user='root',
                                         password='')
        records=[]  
        sql_select_Query=[]
        vihicle=[]
        for i in ids:        
            reqet = "SELECT department_id,company_id,make,model,type_id FROM vehicles WHERE id='" + str(i) + "';"
            sql_select_Query.append(reqet)
       
        for req in sql_select_Query:
            cursor = connection.cursor()
            cursor.execute(req)
            ex=cursor.fetchall()[0]
            msg="this vihicle of the company : "+str(ex[1])+" , departement : "+str(ex[0]) +" with the make : "+ ex[2]+" and a model : "+ex[3]
            records.append(ex)    
            vihicle.append({'message':msg,'type_id':ex[4],"score":scores[get_model(model_name)]})
    except Error as e:
        print("Error reading data from MySQL table", e)
    finally:
        if (connection.is_connected()):
            connection.close()
            cursor.close()
            print("MySQL connection is closed")
    
    return(vihicle)
get_vehicle([4],'DT')       

MySQL connection is closed


[{'message': 'this vihicle of the company : 10 , departement : 3 with the make : Scania and a model : Y6',
  'type_id': 17,
  'score': 0.38636363636363635}]

In [28]:
from flask import Flask, request, make_response
import json
app = Flask(__name__)
@app.route('/', methods=['POST'])
def index():
    
    d = request.form.to_dict() # data recived from form
    date=d['date'].split("T")
    date1=date[0].split("-")
    hours=date[1].split(':')
    date2=list(map(int, date1)) 
    hours2= list(map(int, hours)) 
   
    
    df=pd.DataFrame(columns=['Datatime'],data=np.array(range(len(vihicule_id))))
   
    for i in range(len(vihicule_id)):  
    
        df['Year'] =date1[0]
        df['Month'] =date1[1]
        df['Day'] = date1[2]
        df['Hour'] =int(hours[0])
        
        
    

    xx=pd.concat([df[['Year','Day','Month','Hour']],pd.Series(vihicule_id).rename("id")],sort=False,axis=1) #input data
    result=pd.concat([pd.DataFrame(results[get_model(d['model'])].predict(np.array(xx))),xx],sort=False,axis=1)                 #result of prediction
    result=result.rename(columns={0: 'destination',1:'pickup'})
    ms=result[(result.destination == str( d['destination']).upper()) & (result.pickup == str(d['retour']).upper())] # return only the result of our destination and pickup
        
    if ms.empty:
        vehicles=[{'message': 'there is no vehicles , please change the time or pickup/destination','type_id': -1,'score':scores[get_model(d['model'])]}]
    else:
        
        vehicles= get_vehicle(ms['id'].tolist(),d['model'])       # return the data of vehicles predicted                                                
    
    
   
    
   
    
    print(d)        # data recived from form
    print(result)    # result of the prediction of all vehicles
    print(ms)        # result of the prediction of  vehicle who strat of our pickup and go to destination
    print(vehicles) # the data of vehicle predicted
    
    
    data=make_response(json.dumps(vehicles)) # transform final result to json
    resp=data
    resp.status_code = 200
    resp.headers['Access-Control-Allow-Origin'] = '*' # configiration 
    return resp


    
    
    
if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Oct/2020 09:45:51] "POST / HTTP/1.1" 200 -


{'date': '2020-12-20T10:10', 'retour': 'sfax', 'destination': 'Sousse', 'model': 'SVM'}
   destination pickup  Year Day Month  Hour    id
0        TUNIS  TUNIS  2020  20    12    10  20.0
1        TUNIS  TUNIS  2020  20    12    10  19.0
2        TUNIS  TUNIS  2020  20    12    10   7.0
3        TUNIS  TUNIS  2020  20    12    10   3.0
4        TUNIS  TUNIS  2020  20    12    10   9.0
5        TUNIS  TUNIS  2020  20    12    10  51.0
6        TUNIS  TUNIS  2020  20    12    10  21.0
7        TUNIS  TUNIS  2020  20    12    10  10.0
8        TUNIS  TUNIS  2020  20    12    10  23.0
9        TUNIS  TUNIS  2020  20    12    10  52.0
10       TUNIS  TUNIS  2020  20    12    10  13.0
11       TUNIS  TUNIS  2020  20    12    10  46.0
12       TUNIS  TUNIS  2020  20    12    10  16.0
13       TUNIS  TUNIS  2020  20    12    10  18.0
14       TUNIS  TUNIS  2020  20    12    10  11.0
15       TUNIS  TUNIS  2020  20    12    10   5.0
16       TUNIS  TUNIS  2020  20    12    10  17.0
17       TUN

127.0.0.1 - - [30/Oct/2020 09:46:29] "POST / HTTP/1.1" 200 -


{'date': '2020-12-20T10:10', 'retour': 'sfax', 'destination': 'Sousse', 'model': 'DT'}
   destination  pickup  Year Day Month  Hour    id
0         SFAX   TUNIS  2020  20    12    10  20.0
1       NABEUL    SFAX  2020  20    12    10  19.0
2         SFAX   TUNIS  2020  20    12    10   7.0
3         SFAX   TUNIS  2020  20    12    10   3.0
4        TUNIS    SFAX  2020  20    12    10   9.0
5         SFAX   TUNIS  2020  20    12    10  51.0
6         SFAX   GABES  2020  20    12    10  21.0
7        TUNIS    SFAX  2020  20    12    10  10.0
8         SFAX    BEJA  2020  20    12    10  23.0
9        TUNIS   TUNIS  2020  20    12    10  52.0
10       GABES   TUNIS  2020  20    12    10  13.0
11        SFAX   TUNIS  2020  20    12    10  46.0
12       GABES   TUNIS  2020  20    12    10  16.0
13       GABES   TUNIS  2020  20    12    10  18.0
14       GABES   TUNIS  2020  20    12    10  11.0
15        SFAX   TUNIS  2020  20    12    10   5.0
16       GABES   TUNIS  2020  20    12    10  

...

127.0.0.1 - - [30/Oct/2020 10:01:02] "POST / HTTP/1.1" 200 -


{'date': '2020-12-20T10:10', 'retour': 'sfax', 'destination': 'Sousse', 'model': 'LR'}
   destination  pickup  Year Day Month  Hour    id
0        TUNIS   TUNIS  2020  20    12    10  20.0
1        TUNIS   TUNIS  2020  20    12    10  19.0
2        TUNIS   TUNIS  2020  20    12    10   7.0
3        TUNIS  ARIANA  2020  20    12    10   3.0
4        TUNIS   TUNIS  2020  20    12    10   9.0
5        TUNIS   TUNIS  2020  20    12    10  51.0
6        TUNIS   TUNIS  2020  20    12    10  21.0
7        TUNIS   TUNIS  2020  20    12    10  10.0
8        TUNIS   TUNIS  2020  20    12    10  23.0
9        TUNIS   TUNIS  2020  20    12    10  52.0
10       TUNIS   TUNIS  2020  20    12    10  13.0
11       TUNIS   TUNIS  2020  20    12    10  46.0
12       TUNIS   TUNIS  2020  20    12    10  16.0
13       TUNIS   TUNIS  2020  20    12    10  18.0
14       TUNIS   TUNIS  2020  20    12    10  11.0
15       TUNIS   TUNIS  2020  20    12    10   5.0
16       TUNIS   TUNIS  2020  20    12    10  

127.0.0.1 - - [30/Oct/2020 10:01:40] "POST / HTTP/1.1" 200 -


MySQL connection is closed
{'date': '2020-12-20T10:10', 'retour': 'tunis', 'destination': 'sfax', 'model': 'DT'}
   destination  pickup  Year Day Month  Hour    id
0         SFAX   TUNIS  2020  20    12    10  20.0
1       NABEUL    SFAX  2020  20    12    10  19.0
2         SFAX   TUNIS  2020  20    12    10   7.0
3         SFAX   TUNIS  2020  20    12    10   3.0
4        TUNIS    SFAX  2020  20    12    10   9.0
5         SFAX   TUNIS  2020  20    12    10  51.0
6         SFAX   GABES  2020  20    12    10  21.0
7        TUNIS    SFAX  2020  20    12    10  10.0
8         SFAX    BEJA  2020  20    12    10  23.0
9        TUNIS   TUNIS  2020  20    12    10  52.0
10       GABES   TUNIS  2020  20    12    10  13.0
11        SFAX   TUNIS  2020  20    12    10  46.0
12       GABES   TUNIS  2020  20    12    10  16.0
13       GABES   TUNIS  2020  20    12    10  18.0
14       GABES   TUNIS  2020  20    12    10  11.0
15        SFAX   TUNIS  2020  20    12    10   5.0
16       GABES   TUN

[2020-10-30 10:17:58,294] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-28-9bee8f2a70a1>", line 10, in index
    hours=date[1].split(':')


In [40]:
get_model('SVM')

5
